<a href="https://colab.research.google.com/github/Sirius-Aabhas/CS69002_9A_18CS60R55/blob/master/DL_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import time
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
device = torch.device('cuda:0')
device

##Fetching data

In [0]:
from google.colab import files
import io
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['Train_20K.csv'])


In [0]:
df = pd.read_csv(io.StringIO(uploaded['Train_20K.csv'].decode('utf-8')), sep='\t')
df.head()

##Fetching data for now

In [2]:
df = pd.read_csv('Train_20K.csv', sep='\t')
df.head()

,text,label
0,John Waters has given us a genuinely enjoyable...,1
1,This first two seasons of this comedy series w...,1
2,"What an unfortunate mess is ""Shiner."" I wanted...",0
3,I'm not entirely sure Rob Schmidt qualifies as...,1
4,i wasn't sure whether to laugh or cry. Porrett...,0


In [3]:
df_test = pd.read_csv('Test_5K.csv', sep='\t')
df_test.head()

,text,label
0,as always this is an inaccurate picture of the...,0
1,Did the movie-makers even preview this before ...,0
2,"Heavily re-edited and often confusing, the ori...",0
3,I notice that most of the people who think thi...,0
4,"First of all, this is a low-budget movie, so m...",0


In [4]:
print('Number of Negative movie reviews', len(df[df['label']==0]))
print('Number of Positive movie reviews', len(df[df['label']==1]))

Number of Negative movie reviews 8994
Number of Positive movie reviews 9005


In [5]:
text_reviews = df['text'].astype(str).tolist()
text_labels = df['label'].astype(int)

text_reviews = [x.lower() for x in text_reviews]
print(text_reviews[0], text_labels[0])
print(len(text_reviews), len(text_labels))

john waters has given us a genuinely enjoyable film. this certainly isn't without its shocking waters-esque moments, but it is tamer than his older culty stuff, such as "pink flamingoes". "pecker" harkens back to john's early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in "polyester". overall, a really fun comedy with some great moments! 1
17999 17999


In [6]:
data = [(text_reviews[i], text_labels[i])for i in range(len(text_labels))]
data[:5]

[('john waters has given us a genuinely enjoyable film. this certainly isn\'t without its shocking waters-esque moments, but it is tamer than his older culty stuff, such as "pink flamingoes". "pecker" harkens back to john\'s early mainstream stage in that it reminds the viewer of the same kind of humor that was evident in "polyester". overall, a really fun comedy with some great moments!',
  1),
 ("this first two seasons of this comedy series were very strange and they weren't very funny and had a drama element where bill (the mother) was struggling with all the usual problems in life but that element was a bit depressing and didn't mix well with th comedy elements which is probably why it was dropped. after that it soon became one of the funniest comedy series the bbc have ever made! the chemistry between bill and ben's character's were very funny and there was always so many brilliant and memorable sketches in each series. the christmas specials were hilarious and a real treat for ch

In [0]:
from google.colab import files
import io
uploaded = files.upload()

In [0]:
type(uploaded), uploaded.keys(), type(uploaded['Test_5K.csv'])

In [0]:
df_test = pd.read_csv(io.StringIO(uploaded['Test_5K.csv'].decode('utf-8')), sep='\t')
df_test.head()

In [7]:
print('Number of Negative movie reviews', len(df_test[df_test['label']==0]))
print('Number of Positive movie reviews', len(df_test[df_test['label']==1]))

Number of Negative movie reviews 2482
Number of Positive movie reviews 2518


In [8]:
text_reviews_test = df_test['text'].astype(str).tolist()
text_labels_test = df_test['label'].astype(int)

text_reviews_test = [x.lower() for x in text_reviews_test]
print(text_reviews_test[0], text_labels_test[0])
print(len(text_reviews_test), len(text_labels_test))

as always this is an inaccurate picture of the homeless. tv told a lot of lies about panhandlers in the early 1990s and made everyone look bad, and claimed we all made over $100 a day when $20-40 a day was much closer to reality. when someone drove by where i held up a sign offering to work, and offered me work, i actually went and took the work if i was physically able.and if i would been offered the $100,000 id damned sure invested in in apt prepaid for at least 2 years, and kept most in the bank and still left myself $10-20000 for nl $1-2 and $2-5 cash games at the casinos. i usually always win and could win decent if i just had a bankroll. instead i win about $1000 a month is all playing in always minimum buying in due to not wanting to risk losing it all. i was only homeless cause i didn't wanna risk spending all my money and going broke, sometimes i had over $1000-2000 in my sock while i slept outside. anyone wanting to talk contact sevencard2003 on yahoo messenger.i admit i was 

In [9]:
data_test = [(text_reviews_test[i], text_labels_test[i])for i in range(len(text_labels_test))]
data_test[:5]

[("as always this is an inaccurate picture of the homeless. tv told a lot of lies about panhandlers in the early 1990s and made everyone look bad, and claimed we all made over $100 a day when $20-40 a day was much closer to reality. when someone drove by where i held up a sign offering to work, and offered me work, i actually went and took the work if i was physically able.and if i would been offered the $100,000 id damned sure invested in in apt prepaid for at least 2 years, and kept most in the bank and still left myself $10-20000 for nl $1-2 and $2-5 cash games at the casinos. i usually always win and could win decent if i just had a bankroll. instead i win about $1000 a month is all playing in always minimum buying in due to not wanting to risk losing it all. i was only homeless cause i didn't wanna risk spending all my money and going broke, sometimes i had over $1000-2000 in my sock while i slept outside. anyone wanting to talk contact sevencard2003 on yahoo messenger.i admit i w

## Creating Bag Of Word (BOW) representation of sentences.

In [0]:
def generate_word_ids(dataset):
    word_to_ix = {}
    #print(dataset)
    for sent in dataset:
        for word in sent.split():
            if word not in word_to_ix:
                word_to_ix[word] = len(word_to_ix)
    return word_to_ix

In [23]:
word_to_ix = generate_word_ids(text_reviews + text_reviews_test)
VOCAB_SIZE = len(word_to_ix)
VOCAB_SIZE

239036

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word not in word_to_ix:
            raise ValueError('Word',word,' not present in the dictionary. Sorry!')
        else:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label])


In [17]:
a = torch.ones(1,10)
b = torch.ones(1,10)
a = torch.cat((a,b))
d = torch.ones(1,10)
torch.cat((a,d))


tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [0]:
import torch.utils.data as data_utils

features = torch.tensor(make_bow_vector(text_reviews[0], word_to_ix))
for i in range(1,len(text_reviews)):
    vec = make_bow_vector(text_reviews[i], word_to_ix)
    vec_tensor = torch.tensor(vec)
    features = torch.cat((features,vec_tensor))

targets = torch.tensor(text_labels, dtype=torch.int)

print(features.shape)
print(targets.shape)
train = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train, batch_size=50, shuffle=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


##Task1

In [0]:
class Model1A(nn.Module):
    def __init__(self, vocab_size):
        super(Model1A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.sigmoid(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model1B(nn.Module):
    def __init__(self, vocab_size):
        super(Model1B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.sigmoid(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model1C(nn.Module):
    def __init__(self, vocab_size):
        super(Model1C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.sigmoid(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model1D(nn.Module):
    def __init__(self, vocab_size):
        super(Model1D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 2)

    def forward(self, x):
        x = F.sigmoid(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
model1A = Model1A(VOCAB_SIZE)
model1B = Model1B(VOCAB_SIZE)
model1C = Model1C(VOCAB_SIZE)
model1D = Model1D(VOCAB_SIZE)

In [0]:
for param in model1B.parameters():
    print(param,param.size())

Parameter containing:
tensor([[-8.5557e-04,  9.8191e-04,  1.5383e-03,  ...,  2.1401e-04,
         -1.2505e-03, -1.2147e-03],
        [ 7.9122e-04, -1.6420e-03,  3.8738e-04,  ..., -1.5483e-03,
          2.7893e-04,  1.2792e-03],
        [-7.6434e-04,  1.1936e-03,  1.6871e-04,  ..., -2.6655e-04,
          1.2055e-03,  1.5428e-03],
        ...,
        [ 8.4438e-04, -8.1657e-04,  1.4277e-03,  ...,  5.8581e-04,
          6.7141e-04,  1.1260e-03],
        [ 1.4046e-03,  1.5548e-03, -1.1767e-04,  ..., -1.8034e-03,
         -2.2825e-04,  5.4721e-04],
        [-1.9846e-03, -1.4798e-03, -9.1453e-05,  ...,  2.1273e-05,
          1.6619e-03, -7.0732e-06]], requires_grad=True) torch.Size([50, 239036])
Parameter containing:
tensor([ 4.6513e-04,  6.2723e-04, -1.7822e-03, -3.8633e-04,  1.7975e-03,
        -1.9714e-03, -1.6426e-03,  1.3768e-03, -5.4018e-04, -1.4085e-03,
        -1.6840e-03, -4.3722e-04,  1.1185e-03,  1.7059e-03, -4.1638e-04,
         1.2536e-03, -2.7100e-05, -1.4403e-03, -8.1057e-05, 

##Task2

In [0]:
class Model2A(nn.Module):
    def __init__(self, vocab_size):
        super(Model2A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 10)
        self.lin2 = nn.Linear(10, 10)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model2B(nn.Module):
    def __init__(self, vocab_size):
        super(Model2B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 20)
        self.lin2 = nn.Linear(20, 10)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model2C(nn.Module):
    def __init__(self, vocab_size):
        super(Model2C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 30)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model2D(nn.Module):
    def __init__(self, vocab_size):
        super(Model2D, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 50)
        self.lin2 = nn.Linear(50, 50)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        return F.softmax(self.lin2(x))

In [0]:
class Model2E(nn.Module):
    def __init__(self, vocab_size):
        super(Model2E, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        return F.softmax(self.lin2(x))

##Task3

In [0]:
class Model3A(nn.Module):
    def __init__(self, vocab_size):
        super(Model3A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.lin2 = nn.Linear(100, 50)
        self.lin3 = nn.Linear(50, 10)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        x = F.softmax(self.lin2(x))
        return F.softmax(self.lin3(x))

In [0]:
class Model3B(nn.Module):
    def __init__(self, vocab_size):
        super(Model3B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 200)
        self.lin2 = nn.Linear(200, 100)
        self.lin3 = nn.Linear(100, 10)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        x = F.softmax(self.lin2(x))
        return F.softmax(self.lin3(x))

##Task4

In [0]:
class Model4A(nn.Module):
    def __init__(self, vocab_size):
        super(Model4A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=p)
        self.lin3 = nn.Linear(20, 10)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        x = F.softmax(self.lin2(x))
        x = self.drop_layer(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model4B(nn.Module):
    def __init__(self, vocab_size):
        super(Model4B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=p)
        self.lin2 = nn.Linear(100, 100)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        x = self.drop_layer(x)
        return F.softmax(self.lin2(x))

In [0]:
class Model4C(nn.Module):
    def __init__(self, vocab_size):
        super(Model4C, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 100)
        self.drop_layer = nn.Dropout(p=p)
        self.lin2 = nn.Linear(100, 10)

    def forward(self, x): 
        x = F.softmax(self.lin1(x))
        x = self.drop_layer(x)
        return F.softmax(self.lin2(x))

##Task5

In [0]:
class Model5A(nn.Module):
    def __init__(self, vocab_size):
        super(Model5A, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=p)
        self.lin3 = nn.Linear(20, 10)

    def forward(self, x): 
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.drop_layer(x)
        return F.softmax(self.lin3(x))

In [0]:
class Model5B(nn.Module):
    def __init__(self, vocab_size):
        super(Model5B, self).__init__()
        self.lin1 = nn.Linear(vocab_size, 30)
        self.lin2 = nn.Linear(30, 20)
        self.drop_layer = nn.Dropout(p=p)
        self.lin3 = nn.Linear(20, 10)

    def forward(self, x): 
        x = F.t(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = self.drop_layer(x)
        return F.softmax(self.lin3(x))

## Run code

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(model1A.parameters(), lr = 0.001)

In [0]:
def train(model):
    # the training loop
    for epoch in range(2):
        tic = time.time()
        for instance, label in data_test:
            # get the training data
            model.zero_grad()
            bow_vec = Variable(make_bow_vector(instance, word_to_ix))
            label = make_target(label)
            probs = model(bow_vec) # forward pass
            loss = loss_function(probs, label)
            loss.backward()
            opt.step()
        toc = time.time()
        print('EPOCH: {}, CURRENT LOSS: {}, TIME TAKEN: {}'.format(epoch, loss.data, (toc-tic)))


In [0]:
train(model1A)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':


EPOCH: 0, CURRENT LOSS: -0.4067547023296356, TIME TAKEN: 306.377108335495
EPOCH: 1, CURRENT LOSS: -0.4067547023296356, TIME TAKEN: 612.4105689525604


In [0]:
def validate(model):
    for instance, label in data_test:
        bow_vec = Variable(make_bow_vector(instance, word_to_ix))
        logprobs = model(bow_vec)
        #print(logprobs)
        pred = np.argmax(logprobs.data.numpy())
        print('prediction: {}'.format(pred))
        print('actual: {}'.format(label))

In [0]:
validate(model1A)

In [0]:
torch.save(model1A, 'model1A.mdl')

In [0]:
mdl_1a = torch.load('model1A.mdl')

In [0]:
validate(mdl_1a)